In [3]:
from mxnet import *
from mxnet.gluon import nn
npx.set_np()



In [6]:
x = np.random.uniform(size=(2,20))

net = nn.Sequential()
net.add(nn.Dense(256, activation='relu'))
net.add(nn.Dense(10))

net.initialize()
net(x)

array([[ 0.02932412, -0.06546377, -0.07086012,  0.03554048,  0.0082543 ,
         0.03487837,  0.00308158,  0.02359925,  0.0744891 , -0.0402443 ],
       [ 0.01776894, -0.08951396, -0.09595288,  0.0497277 ,  0.03439742,
         0.05931327,  0.05047961, -0.02034373,  0.0067703 ,  0.0009637 ]])

## CUSTOM BLOCK

In [7]:
class MLP(nn.Block):
    
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Dense(256, activation='relu')
        self.output = nn.Dense(10)
        
    def forward(self, x):
        return self.output(self.hidden(x))
        

In [8]:
net = MLP()
net.initialize()
net(x)

array([[-0.06719264, -0.0543559 , -0.01983661, -0.07464816, -0.07466961,
         0.05775826,  0.10255943,  0.04728728,  0.01342697,  0.01783174],
       [-0.07974197, -0.06915514, -0.0627503 , -0.06871963,  0.00517473,
         0.03918917,  0.05325559,  0.05016641, -0.04484216,  0.05205344]])

## SEQUENTIAL BLOCK

In [9]:
class MySequential(nn.Block):
    def add(self, block):
        self._children[block.name] = block
    
    def forward(self, x):
        for block in self._children.values():
            x = block(x)
        return x

In [11]:
net = MySequential()
net.add(nn.Dense(256, activation= 'relu'))
net.add(nn.Dense(10))
net.initialize()
net(x)

array([[-0.01329408,  0.04887484, -0.00750186,  0.02435913, -0.01234759,
        -0.03700005,  0.01387178,  0.02878573,  0.04961637, -0.00815793],
       [ 0.0372449 ,  0.016038  , -0.01329308,  0.0091404 , -0.0341957 ,
        -0.0474561 , -0.01623891,  0.03003193,  0.00446071,  0.03678473]])

## EXECUTE FORWARD 

In [12]:
class FixedHiddenMLP(nn.Block):
    
    def __init__(self, **kwargs):
        super(FixedHiddenMLP, self).__init__(**kwargs)
        
        self.rand_weight = self.params.get_constant(
        'rand_weight', np.random.uniform(size=(20, 20)))
        
        self.dense = nn.Dense(20, activation='relu')
        
    def forward(self, x):
        x = self.dense(x)
        x = npx.relu(np.dot(x, self.rand_weight.data()) +1)
        x = self.dense(x)
        
        while np.abs(x).sum() > 1:
            x /= 2
        
        return x.sum()

In [13]:
net = FixedHiddenMLP()
net.initialize()
net(x)

array(0.91638494)

In [14]:
class NestMLP(nn.Block):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        
        self.net = nn.Sequential()
        self.net.add(nn.Dense(64, activation= 'relu'),
                    nn.Dense(32, activation= 'relu'))
        
        self.dense = nn.Dense(16, activation= 'relu')
    
    def forward(self, x):
        return self.dense(self.net(x))

In [15]:
chimera = nn.Sequential()
chimera.add(NestMLP(), nn.Dense(20), FixedHiddenMLP())

chimera.initialize()
chimera(x)

array(0.66665936)